In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from plotly import tools, io
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import math
from scipy.interpolate import interp1d

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

from numpy.linalg import norm

from scipy.optimize import fmin

In [ ]:
prefix = 'xxx' # set the folder used with CSV logs
filename = 'dots-detections.csv'
max_iteration = 50
export_html = True

# load csv data
data_log = pd.read_csv(prefix+filename,header=0, index_col=False)
data_log['t'] = (data_log['frame_id']-data_log['frame_id'][0])

In [ ]:
# show the position data diagram
p=[]           
for c in ['real_x','real_y']:    
    trace=go.Scatter(
        y= data_log[c],
        x= data_log.t/1000,
        name = c,
        mode='markers'
    )
    p.append(trace)
    
for c in ['expected_x','expected_y']:    
    trace=go.Scatter(
        y= data_log[c],
        x= data_log.t/1000,
        name = c,
#         mode='markers'
    )
    p.append(trace)
                  
axis_template=dict(
    showgrid=True,
    zeroline=True,
    nticks=20,
    showline=True,
    zerolinecolor="#FF0000"
)
layout=go.Layout(
    xaxis=axis_template,
    yaxis=axis_template,
    title='original position log'
)

fig=go.Figure(
    data=p,
    layout=layout
)
fig.update_layout({'xaxis':{'title':'time in s'}})
iplot(fig)
if export_html:
    io.write_html(fig, include_plotlyjs='cdn',file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')


In [ ]:
# show the visual & real position error  in pixel
data_diff_x = np.subtract(data_log['real_x'],data_log['expected_x'])
data_diff_y = np.subtract(data_log['real_y'],data_log['expected_y'])

p=[]
#index = np.arange(0,len(data_log))
# index = data_log['frame_id']

p.append(go.Scatter(
    y= data_diff_x,
    x= data_log.t/1000,
    name='diff x in pixel'
    
))


p.append(go.Scatter(
    y= data_diff_y,
    x= data_log.t/1000,
    name='diff y in pixel'
))


layout.title= 'position error in pixel'
fig=go.Figure(
    data=p,
    layout=layout
)
iplot(fig)
# if export_html:
#     io.write_html(fig,include_plotlyjs='cdn', file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')

In [ ]:
# get the uv error digram
mark_width = 250 #250mm
mark_height = 150 #150mm
expected_u = np.mean(data_log['u'][0:10])
expected_v = np.mean(data_log['v'][0:10])
real_u = data_log['u']
real_v = data_log['v']

data_diff_u_dist = np.subtract(real_u,expected_u)*mark_width
data_diff_v_dist = np.subtract(real_v,expected_v)*mark_height

p=[]
#index = np.arange(0,len(data_log))
# index = data_log['frame_id']

p.append(go.Scatter(
    y= data_diff_u_dist,
    x= data_log.t/1000,
    name='diff x in mm'    
))
p.append(go.Scatter(
    y= data_diff_v_dist,
    x= data_log.t/1000,
    name='diff y in mm'
))
layout.title= 'position error in mm'
fig=go.Figure(
    data=p,
    layout=layout
)
layout.xaxis['title'] = 'frames'
layout.yaxis['title'] = 'position error in mm'
iplot(fig)
if export_html:
    io.write_html(fig,include_plotlyjs='cdn', file=prefix+filename.replace('.csv','_')+layout.title.text.replace(' ','_')+'.html')

In [ ]:
def latency(data, axis='x'):
    tmp = data
    tmp.reset_index()
    def dt(x):
        t = x[0]
        b = x[1]
        a = x[2]
        tmp_diff = data['real_'+axis]-np.interp(data.t-t, data.t, tmp['expected_'+axis]*a+b)
        return  np.linalg.norm(tmp_diff, axis=0).mean()
    return fmin(dt, [10, 0., 1.])

def show_data_log(data_log,x=[0.,0.,1.],name=''):
    delay = x[0]
    b = x[1]
    a = x[2]
    p=[]           
    for c in ['real_x','real_y']:    
        trace=go.Scatter(
            y= data_log[c],
            x= data_log.t/1000,
            name = c,
            mode='markers'
        )
        p.append(trace)
    for c in ['expected_x','expected_y']:    
        trace=go.Scatter(
            y= data_log[c]*a+b,
            x= data_log.t/1000+delay/1000,
            name = c,
            mode='lines'
        )
        p.append(trace)
    axis_template=dict(
        showgrid=True,
        zeroline=True,
        nticks=20,
        showline=True,
        zerolinecolor="#FF0000"
    )
    layout=go.Layout(
        xaxis=axis_template,
        yaxis=axis_template,
        title='delay {:.2f}ms {}'.format(delay, name)
    )
    fig=go.Figure(data=p,layout=layout)
    fig.update_layout({'xaxis':{'title':'time in s'}})
    iplot(fig)
    if export_html:
        io.write_html(fig,include_plotlyjs='cdn', file=prefix+filename.replace('.csv','_')+"delay_"+name+'.html')

t0 = 4000;
t1 = t0+7000
latencies = []
        
idx = (data_log.t > t0) & (data_log.t < t1)
res = latency(data_log[idx], 'x')
delay = res[0]
b = res[1]
a = res[2]
print("Delay yaw {} ms ({} {})".format(delay, b, a))
latencies.append([delay,a,b])
show_data_log(data_log[idx], res, "yaw")

t0 = t1 + 500
t1 = t0 +7000
idx = (data_log.t > t0) & (data_log.t < t1)
res = latency(data_log[idx], 'y')
delay = res[0]
b = res[1]
a = res[2]
print("Delay pitch {} ms ({} {})".format(delay, b, a))
latencies.append([delay,a,b])
show_data_log(data_log[idx], res, "pitch")

t0 = t1+7000 + 500 +500
t1 = t0 +7000
idx = (data_log.t > t0) & (data_log.t < t1)
res = latency(data_log[idx], 'x')
delay = res[0]
b = res[1]
a = res[2]
print("Delay trans X {} ms ({} {})".format(delay, b, a))
latencies.append([delay,a,b])
show_data_log(data_log[idx], res, "x_trans")

t0 = t1 + 500
t1 = t0 +7000
idx = (data_log.t > t0) & (data_log.t < t1)
res = latency(data_log[idx], 'y')
delay = res[0]
b = res[1]
a = res[2]
print("Delay trans Y {} ms ({} {})".format(delay, b, a))
latencies.append([delay,a,b])
show_data_log(data_log[idx], res, "y_trans")


for i in [0,1,2,3]:
    print("{:.3f}\t{:.3f}\t{:.3f}".format(latencies[i][0],latencies[i][2],latencies[i][1]))
